In [1]:
# Each line after the first is either broad, medium, or narrow.
# Also sum for each industry.
# For the first attempt, just ignore the non-national data.
# Descending order of size: Major, Minor, Broad, Detail.
import csv
with open("test_data.csv",'r') as f:
    c=csv.reader(f)
    l=list(c)
print(l[0])

['area', 'area_title', 'area_type', 'naics', 'naics_title', 'own_code', 'occ code', 'occ title', 'group', 'tot_emp', 'emp_prse', 'jobs_1000', 'loc_quotient', 'pct_total', 'h_mean', ' a_mean ', 'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90', ' a_pct10 ', ' a_pct25 ', ' a_median ', ' a_pct75 ', ' a_pct90 ', 'annual', 'hourly']


In [5]:
d=dict()
def get(i,count):
    return [j[count] for j in i[1:]]
d[l[0][3]]= get(l,3)  #naics
d[l[0][6]]= get(l,6)  #occ code
d[l[0][8]]= get(l,8)  #Group
d[l[0][9]]= get(l,9)  #Total employment
d[l[0][14]]=get(l,14) #Mean Hourly Pay
d[l[0][15]]=get(l,15) #Mean Annual Pay
d["h_pay_dist"]=get(l,slice(17,22))
d["a_pay_dist"]=get(l,slice(22,27))

d.keys()

dict_keys(['naics', 'occ code', 'group', 'tot_emp', 'h_mean', ' a_mean ', 'h_pay_dist', 'a_pay_dist'])